In [24]:
# IMPORT NECESSARY MODULES
import os
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import heapq
import math

folder_path = "D:\\UPD Files\\BS CS 24-25 1st Sem\\CS 198\\cs198-protein-folding\\datasets"

In [ ]:
# Dijkstra with wrap-around on a 3D surface (NxN matrix) with heights
def dijkstra_wrap_around_surface(surface, start, goal):
    rows, cols = len(surface), len(surface[0])
    
    # Directions: Up, Down, Left, Right, and 4 Diagonals
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), 
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]  # Including diagonals

    def get_neighbors(node):
        x, y = node
        neighbors = []
        for dx, dy in directions:
            nx, ny = (x + dx) % rows, (y + dy) % cols  # Wrap-around using modulo
            neighbors.append((nx, ny))
        return neighbors
    
    def distance(current, neighbor):
        # Euclidean distance between two points, considering height difference
        x1, y1 = current
        x2, y2 = neighbor
        dx = min(abs(x1 - x2), rows - abs(x1 - x2))
        dy = min(abs(y1 - y2), cols - abs(y1 - y2))
        dz = abs(surface[x1][y1] - surface[x2][y2])
        return math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

    open_list = []
    heapq.heappush(open_list, (0, start))  # Priority queue (f-cost, node)
    came_from = {}
    g_score = {start: 0}
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        if current == goal:
            # Reconstruct the path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            print("Total distance:", g_score[goal])
            return path
        
        for neighbor in get_neighbors(current):
            tentative_g_score = g_score[current] + distance(current, neighbor)
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score
                heapq.heappush(open_list, (f_score, neighbor))
    
    return None  # Return None if no path is found

if __name__ == "__main__":
    # Example usage:
    # A 5x5 grid where each value represents a height (z-coordinate) of a 3D surface
    surface = [
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [1, 2, 3, 4, 5]
    ]
    start = (0, 0)
    goal = (4, 4)
    path = dijkstra_wrap_around_surface(surface, start, goal)
    print("Path found:", path)

Total distance: 0
Path found: [(0, 0), (4, 4)]


In [26]:
# A* with wrap-around on a 3D surface (NxN matrix) with heights
def a_star_wrap_around_surface(surface, start, goal):
    rows, cols = len(surface), len(surface[0])
    
    # Directions: Up, Down, Left, Right, and 4 Diagonals
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), 
                  (-1, -1), (-1, 1), (1, -1), (1, 1)]  # Including diagonals

    def get_neighbors(node):
        x, y = node
        neighbors = []
        for dx, dy in directions:
            nx, ny = (x + dx) % rows, (y + dy) % cols  # Wrap-around using modulo
            neighbors.append((nx, ny))
        return neighbors

    def heuristic(node1, node2):
        x1, y1 = node1
        x2, y2 = node2
        h1 = surface[x1][y1]
        h2 = surface[x2][y2]
        # Wrap-around Euclidean distance heuristic, considering height
        dx = min(abs(x1 - x2), rows - abs(x1 - x2))
        dy = min(abs(y1 - y2), cols - abs(y1 - y2))
        dz = abs(h1 - h2)
        return math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

    def distance(current, neighbor):
        # Euclidean distance between two points, considering height difference
        x1, y1 = current
        x2, y2 = neighbor
        dx = min(abs(x1 - x2), rows - abs(x1 - x2))
        dy = min(abs(y1 - y2), cols - abs(y1 - y2))
        dz = abs(surface[x1][y1] - surface[x2][y2])
        return math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

    open_list = []
    heapq.heappush(open_list, (0, start))  # Priority queue (f-cost, node)
    came_from = {}
    g_score = {start: 0}
    
    while open_list:
        _, current = heapq.heappop(open_list)
        
        if current == goal:
            # Reconstruct the path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            print("Total distance:", g_score[goal])
            return path
        
        for neighbor in get_neighbors(current):
            tentative_g_score = g_score[current] + distance(current, neighbor)
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_list, (f_score, neighbor))
    
    return None  # Return None if no path is found


if __name__ == "__main__":
    # Example usage:
    # A 5x5 grid where each value represents a height (z-coordinate) of a 3D surface
    surface = [
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [1, 2, 3, 4, 5]
    ]
    start = (0, 0)
    goal = (4, 4)
    path = a_star_wrap_around_surface(surface, start, goal)
    print("Path found:", path)

Total distance: 4.242640687119285
Path found: [(0, 0), (4, 4)]
